# PRAKTIKUM 2
### Generator Teks dengan RNN

Praktikum ini mendemonstrasikan cara melakukan genearsi text menggunakan RNN. Dataset yang digunkan adalah dataset Shakespeare's writing from Andrej Karpathy's The Unreasonable Effectiveness of Recurrent Neural Networks. Jika diberikan urutan karakter dari data ini ("Shakespear"), latih model untuk memprediksi karakter berikutnya dalam urutan ("e"). Urutan teks yang lebih panjang dapat dihasilkan dengan memanggil model berulang kali.


Tutorial ini menggunakan tf.keras dan eager execution. Berikut adalah contoh output ketika model dalam tutorial ini dilatih selama 30 epoch, dan dimulai dengan prompt "Q":

### Setup
#### Import Tensorflow

In [26]:
import tensorflow as tf
import numpy as np
import os
import time

#### Download Dataset Shakespeare

In [27]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


#### Load Data

In [28]:
# Read, the people decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding = 'utf-8')

# length of text is the number characters in it
print(f'Length of text : {len(text)}characters')

Length of text : 1115394characters


In [29]:
# Take a look at the first 250 characters in text
print(text[:250])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [30]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


### Olah Teks
#### Vectorize Teks
Sebelum training, Anda perlu mengonversi string menjadi representasi numerik. tf.keras.layers.StringLookup dapat mengubah setiap karakter menjadi ID numerik. Caranya adalah teks akan dipecah menjadi token terlebih dahulu.

In [31]:
# Membagi teks menjadi karakter-karakter Unicode
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

sekarang buat tf.keras.layers.StringLookup layer:

In [32]:
# Membuat layer StringLookup untuk mengonversi karakter menjadi ID
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)


perintah diatas mengconvert token menjadi id

In [33]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Karena tujuan tutorial ini adalah untuk menghasilkan teks, penting juga untuk membalikkan representasi ini. Untuk ini Anda dapat menggunakan kode tf.keras.layers.StringLookup(..., invert=True).
Catatan: pada kode ini, daripada meneruskan kosakata asli yang dihasilkan dengan diurutkan(set(teks)) gunakan metode get_vocabulary() dari tf.keras.layers.StringLookup sehingga token [UNK] disetel dengan cara yang sama.

In [34]:
# Membuat layer StringLookup untuk mengonversi ID menjadi karakter
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True, mask_token=None
)


Lapisan ini mengconvert kembali karakter dari vektor ID, dan mengembalikannya sebagai karakter tf.RaggedTensor:

In [35]:
# Mengonversi ID kembali menjadi karakter
chars = chars_from_ids(ids)
chars


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Anda dapat menggunakan tf.strings.reduce_join untuk menggabungkan kembali karakter menjadi string.

In [36]:
# Menggabungkan karakter menjadi string
tf.strings.reduce_join(chars, axis=-1).numpy()


array([b'abcdefg', b'xyz'], dtype=object)

In [37]:
# Fungsi untuk mendapatkan teks dari ID
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


### Prediksi
#### Membuat Trianing Set dan Target

Selanjutnya bagilah teks menjadi contoh sequence. Setiap masukan sequence akan berisi karakter seq_length dari teks. Untuk setiap masukan sequence, target prediksi berisi teks dengan panjang yang sama, hanya digeser satu karakter ke kanan. Jadi, bagi teks menjadi beberapa bagian seq_length+1. Misalnya, seq_length adalah 4 dan teks kita adalah "Hello". Urutan masukannya adalah "Hell", dan urutan targetnya adalah "ello". Untuk melakukan ini, pertama-tama gunakan fungsi tf.data.Dataset.from_tensor_slices untuk mengonversi vektor teks menjadi aliran indeks karakter.

In [38]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [39]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [40]:
# Menampilkan teks dari 10 batch pertama dalam dataset ID
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('utf-8'))


F
i
r
s
t
 
C
i
t
i


In [41]:
seq_length = 100

Metode batch memungkinkan Anda dengan mudah mengonversi karakter individual ini menjadi urutan ukuran yang diinginkan.

In [42]:
# Membuat urutan karakter dari dataset ID dengan panjang sekuens ditambah 1
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# Menampilkan urutan karakter dari 1 batch pertama
for seq in sequences.take(1):
  print(chars_from_ids(seq))


tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


akan lebih mudah untuk melihat apa yang dilakukan jika Anda menggabungkan token kembali menjadi string:

In [43]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [44]:
# Fungsi untuk memisahkan input dan target dari sebuah urutan karakter
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text


In [45]:
# Contoh penggunaan fungsi split_input_target pada urutan karakter "Tensorflow"
split_input_target(list("Tensorflow"))


(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [46]:
# Membuat dataset baru dengan memetakan fungsi split_input_target pada setiap elemen urutan karakter
dataset = sequences.map(split_input_target)


In [47]:
# Mengambil satu contoh dari dataset hasil pemisahan input dan target
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())


Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


#### Membuat Batch Training
Anda menggunakan tf.data untuk membagi teks menjadi sequence yang dapat diatur. Namun sebelum memasukkan data ini ke dalam model, Anda perlu mengacak data dan mengemasnya ke dalam batch.


In [48]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### Buat Model
Bagian ini mendefinisikan model sebagai subkelas keras.Model (untuk lebih detilnya, lihat Making new Layers and Models via subclassing).
Model yang kita bangun memiliki 3 lapisan neural network :
- tf.keras.layers.Embedding: Lapisan masukan. Tabel pencarian yang dapat dilatih yang akan memetakan setiap karakter-ID ke vektor dengan dimensi embedding_dim;
- tf.keras.layers.GRU: lapisan RNN dengan ukuran unit=rnn_units (Anda juga dapat menggunakan lapisan LSTM di sini.)
- tf.keras.layers.Dense: Lapisan keluaran, dengan keluaran vocab_size. Ini menghasilkan satu logit untuk setiap karakter dalam kosakata. Ini adalah log kemungkinan setiap karakter menurut model.

In [49]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [50]:
# Definisi kelas model kustom MyModel
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x


In [51]:
# Membuat instance model kustom MyModel
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


### Uji Model
pertama, cek bentuk dari output

In [52]:
# Melakukan inferensi untuk satu batch contoh dari dataset
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 66) # (batch_size, sequence_length, vocab_size)


Dalam contoh di atas, panjang urutan masukan adalah 100 tetapi model dapat dijalankan pada masukan dengan panjang berapa pun:

In [53]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense_4 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
# Menggunakan distribusi kategorikal untuk menghasilkan indeks yang diambil secara acak
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# Menyusun kembali bentuk tensor dan mengonversinya ke dalam array NumPy
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()


In [55]:
sampled_indices

array([61, 12,  9, 13, 27, 48, 58, 62,  9, 22,  8, 49, 10, 52, 28, 22, 16,
       12, 39, 26, 25, 27, 50, 41, 55, 54, 49, 33, 30,  1, 46, 33, 39, 27,
       30, 54, 14, 49, 59, 30, 48, 20, 20, 21, 42, 63, 62,  7, 42, 29,  1,
        0, 32, 29, 16, 16,  9, 39, 64,  8, 41, 32, 27,  1, 13, 28, 46, 30,
        2, 44, 59, 58,  6, 22, 13, 34, 22, 34, 43,  4, 65,  6, 31, 34, 20,
       23, 15, 42, 59, 46, 52,  3, 42, 52,  3, 57,  7, 26, 47, 34])

Dekode kode berikut untuk melihat teks yang diprediksi oleh model tidak terlatih ini:

In [56]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'd to wear a crown,\nHis hand to wield a sceptre, and himself\nLikely in time to bless a regal throne.\n'

Next Char Predictions:
 b"v;.?Nisw.I-j3mOIC;ZMLNkbpojTQ\ngTZNQoAjtQiGGHcxw,cP\n[UNK]SPCC.Zy-bSN\n?OgQ ets'I?UIUd$z'RUGJBctgm!cm!r,MhU"


### Train Model
#### Tambahan optimizer dan fungsi loss
loss function tf.keras.losses.sparse_categorical_crossentropy standar berfungsi dalam kasus ini karena diterapkan di seluruh dimensi terakhir prediksi. Karena model Anda mengembalikan logits, Anda perlu mengatur flag from_logits.

In [57]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


In [58]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190126, shape=(), dtype=float32)


In [59]:
tf.exp(example_batch_mean_loss).numpy()

66.031105

In [60]:
model.compile(optimizer='adam', loss=loss)

#### Konfigurasi Checkpoints

In [61]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### Lakukan Proses Training



In [62]:
EPOCHS = 20

In [63]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 13s 56ms/step - loss: 2.7280
Epoch 2/20
172/172 [==============================] - 11s 55ms/step - loss: 1.9937
Epoch 3/20
172/172 [==============================] - 12s 55ms/step - loss: 1.7175
Epoch 4/20
172/172 [==============================] - 12s 56ms/step - loss: 1.5556
Epoch 5/20
172/172 [==============================] - 12s 56ms/step - loss: 1.4561
Epoch 6/20
172/172 [==============================] - 11s 56ms/step - loss: 1.3866
Epoch 7/20
172/172 [==============================] - 11s 56ms/step - loss: 1.3333
Epoch 8/20
172/172 [==============================] - 12s 57ms/step - loss: 1.2869
Epoch 9/20
172/172 [==============================] - 13s 57ms/step - loss: 1.2457
Epoch 10/20
172/172 [==============================] - 13s 58ms/step - loss: 1.2057
Epoch 11/20
172/172 [==============================] - 12s 58ms/step - loss: 1.1666
Epoch 12/20
172/172 [==============================] - 12s 58ms/step - loss: 1.1256
E

### Generate Teks


In [64]:

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [65]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)


In [66]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
How brook it not?

BRUTUS:
Consild his presence, my lord.

DUKE VINCENTIO:
What's the soul of thy mother, nay, nor if he?

QUEEN ELIZABETH:
Say not, sir, general; come, sir, ask'd am: if he swore,
Your purish in this. But that you love your house,
A graver increase that he is held
But one a fool thy bride royal times,
Drawn say the seas on them; I had thought
so criminal to his curse intends;
And you were not prepared for an hour fortune.

KING RICHARD III:
Blessed bewith my sworn, lords, good Kate,
And take durst no longer stake a vice: only things not smake
To know not at your very librer prison.

RICHARD:
Thou art poor in, and much in whood
To answer it.

AUFIDIUS:
I know it.

GRUMIO:
King Edward, voice and these good request,
For did ner two waits upon her, for the extremest of thine ears;
Believe it, and our friends substiture's in your extremest.

JULIET:
You are too honest to do instance.

DUKE VINCENTIO:
I thank your grace.

GLOUCESTER:
Here's Pomp your death.

First Sen

In [67]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nWhat say you, sir?\n\nROMEO:\nWhy, have you faintly conquered your honour never.\nBut feact from me, let that move me no excuse?\n\nAUTOLYCUS:\nI am Green in heaven; but I'll think thou hadst, though I not do't.\n\nCORIOLANUS:\nVincet it but a pit o' the mirth,\nThan pity not what like you one would forget.\n\nNORTHUMBERLAND:\nNay, I think he is before to know his promises,\nWho hath pass'd and straight of lamentation.\nGo, ready throw at this I came.\n\nTRANIO:\nI know will then?\n\nNurse:\nFie, young aprous will to frame mine oath.\n\nThird Citizen:\nThere is a traitor, can you lay they live,\nThat shall be hated that husband me,\nThat he did business send upon thy yate;\nBut that hath crim'd the torch to wail their wrath;\nNor that which lives like a good world\nBuy, while my father now it do:\nHis prayers and undertaking Tybalt demandry\nAre well contradiction:\nTo earn to Lord Angelo is cross, now but\nLetin's weakness with maintesher, to stande,\nAnd therefo

### Ekspor Model Generator


In [68]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [69]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))


ROMEO:
I should have knowledge for that King Lewis
Because to be my pent to supply the roor;
For had I lau


In [70]:
class CustomTraining(MyModel):
 @tf.function
 def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
        predictions = self(inputs, training=True)
        loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [71]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [72]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [73]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 15s 61ms/step - loss: 2.7255


In [74]:

EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
      logs = model.train_step([inp, target])
      mean.update_state(logs['loss'])

      if batch_n % 50 == 0:
         template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
         print(template)

 # saving (checkpoint) the model every 5 epochs
      if (epoch + 1) % 5 == 0:
         model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1640
Epoch 1 Batch 50 Loss 2.0604
Epoch 1 Batch 100 Loss 1.9571
Epoch 1 Batch 150 Loss 1.8771

Epoch 1 Loss: 1.9983
Time taken for 1 epoch 12.80 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8489
Epoch 2 Batch 50 Loss 1.7650
Epoch 2 Batch 100 Loss 1.7357
Epoch 2 Batch 150 Loss 1.6363

Epoch 2 Loss: 1.7132
Time taken for 1 epoch 11.81 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5900
Epoch 3 Batch 50 Loss 1.5626
Epoch 3 Batch 100 Loss 1.5091
Epoch 3 Batch 150 Loss 1.5381

Epoch 3 Loss: 1.5503
Time taken for 1 epoch 12.59 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4876
Epoch 4 Batch 50 Loss 1.4260
Epoch 4 Batch 100 Loss 1.4630
Epoch 4 Batch 150 Loss 1.4686

Epoch 4 Loss: 1.4504
Time taken for 1 epoch 12.91 sec
_____________________________________________________________________